In [3]:
import pandas as pd
import glob
import scipy.io as io
import numpy
from tfcat import TFCat
from shapely.geometry import MultiPoint, Point, Polygon, MultiPolygon
from astropy.time import Time

import matplotlib.pyplot as plt
import matplotlib.cm as cm

from matplotlib.colors import LinearSegmentedColormap
import matplotlib.colors as colors
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits import axes_grid1
from matplotlib.ticker import (MultipleLocator,
                               FormatStrFormatter,
                               AutoMinorLocator)


import h5py
from h5py import File
import numpy as np
from datetime import datetime
from os import path
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import csv
from tqdm import tqdm
import seaborn as sns 
from scipy.stats import binned_statistic_2d
from shapely.ops import unary_union

import plotly.graph_objects as go
plt.rcParams["figure.figsize"] = (200,100)


/Users/clouis/opt/anaconda3/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.1)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [4]:
def load_hdf5_data(file_name: str):
    """
    Reads a datafile in HDF5 format using the config file set earlier.
    """
    file: File = File(file_name)
    time = file['Time']
    freqs = file['Frequency']
    data = file['Data']
    return time, freqs, data

In [24]:
def polar_ephemeris_load(file, time_view_start, time_view_end, coordinate_system = "GSM"):
    df = pd.read_csv(file, delimiter = " ", header = None, skiprows=4,
                    names=["Date", "Time", "x_geo", "y_geo", "z_geo", "GEO_LAT", "GEO_LONG", "x_gm", "y_gm", "z_gm", "GM_LAT", "GM_LONG", "x_gse", "y_gse", "z_gse", "GSE_LAT", "GSE_LONG", "LT_gsm", "x_gsm", "y_gsm", "z_gsm", "GSM_LAT", "GSM_LONG", "Distance_R_E"])
    df["Date"] = "19"+df["Date"] 
    df.index = pd.to_datetime(df["Date"]+"T"+df["Time"])
    df.drop(["Date", "Time"], axis = 1, inplace = True)

    #Restricting flux and ephemeris data to be within the user defined start/end times
    
    df = df[(df.index > time_view_start) & (df.index < time_view_end)]

    date = numpy.array(Time(df.index).unix)
    latitude = numpy.array(df[coordinate_system+"_LAT"])
    longitude = numpy.array(df[coordinate_system+"_LONG"])
    distance = numpy.array(df["Distance_R_E"])
    return(date,latitude,longitude, distance)

In [5]:
def polar_ephemeris_load_xyz(file, time_view_start, time_view_end, coordinate_system = "GSM"):
    df = pd.read_csv(file, delimiter = " ", header = None, skiprows=4,
                    names=["Date", "Time", "x_geo", "y_geo", "z_geo", "GEO_LAT", "GEO_LONG", "x_gm", "y_gm", "z_gm", "GM_LAT", "GM_LONG", "x_gse", "y_gse", "z_gse", "GSE_LAT", "GSE_LONG", "LT_gsm", "x_gsm", "y_gsm", "z_gsm", "GSM_LAT", "GSM_LONG", "Distance_R_E"])
    df["Date"] = "19"+df["Date"] 
    df.index = pd.to_datetime(df["Date"]+"T"+df["Time"])
    df.drop(["Date", "Time"], axis = 1, inplace = True)

    #Restricting flux and ephemeris data to be within the user defined start/end times
    
    df = df[(df.index > time_view_start) & (df.index < time_view_end)]

    date = numpy.array(Time(df.index).unix)
    x = numpy.array(df["x_"+coordinate_system.lower()], dtype=float)
    y = numpy.array(df["y_"+coordinate_system.lower()], dtype=float)
    z = numpy.array(df["z_"+coordinate_system.lower()], dtype=float)

    return(date,x,y,z)

In [22]:
def plot_trajectory_magnetic_equatorial_plane(rho_orbit,z_orbit, flux_orbit,coordinate_system,  save_html = False, save_png = False, show_plot = True):

    cmax_orbit =  np.quantile(flux_orbit,0.90)
    cmin_orbit = np.quantile(flux_orbit,0.35)

    fig = go.Figure(data=go.Scatter(
                    x=rho_orbit,
                    y=z_orbit,
                    mode='markers',
                    marker=dict(
                        size=2,
                        color=flux_orbit,
                        cmax = cmax_orbit,
                        cmin = cmin_orbit,
                        colorbar = dict(
                            title = "AKR Intensity"
                        ),
                        colorscale='Viridis',
                        opacity = 0.8
                    ), 
        showlegend = False,
        ))
        
    fig.update_xaxes(range=[-6,4], title="X ("+coordinate_system+")")
    fig.update_yaxes(range=[-2,10],title = "Y ("+coordinate_system+")")
    fig.update_layout(font = dict(size = 15))

    fig.add_shape(type="circle",
                    xref="x", yref="y",
                    fillcolor="Grey",
                    x0=-1, y0=-1, x1=1, y1=1,
                    line_color="Grey",
                    )
    
    if save_png == True:
        fig.write_image("polar_trajectory_magnetic_equatorial_plane_"+coordinate_system+".pdf")
    if show_plot == True:
        fig.show()
    if save_html == True:
        fig.write_html("polar_trajectory_magnetic_equatorial_plane_"+coordinate_system+".html") #Modifiy the html file
    

    return

In [7]:
def plot_colourline(x, y, c, vmin=None, vmax=None, colormap=cm.viridis):
    if vmin == None:
        vmin = np.quantile(c,0.35)
    if vmax == None:
        vmax = np.quantile(c,0.90)

    intensity = c.copy()
    intensity[c == -1] = vmin

    # color = colormap(c / (vmax - vmin))
    color = colormap(colors.LogNorm(vmin=vmin, vmax=vmax)(intensity))
    ax = plt.gca()
    for i in np.arange(len(x) - 1):
        ax.plot([x[i], x[i + 1]], [y[i], y[i + 1]], c=color[i])
    return


In [8]:
#path to ephemeris and save directory:
dir_save_name = "/Users/clouis/Documents/Data/POLAR/data"
#Polar ephemeris data taken from https://sscweb.gsfc.nasa.gov/cgi-bin/Locator.cgi
data_path_ephemeris = glob.glob("/Users/clouis/Documents/Data/POLAR/data/polar_ephemeris_geo_gm_gse_gsm.txt")[0]

time_view_start_all_orbit = '1996-03-25'
time_view_end_all_orbit ='1996-05-03'

i_coordinate_system=0

#coordinate_system_table = ["GSE", "GEO", "GSM","GM"]
coordinate_system_table = ["GSM"]

type = ["all"]
type_feature = "ALL"
path_masked_data = "/Users/clouis/Documents/Data/POLAR/data/polar_all_features_mask.hdf5"

 # load masked data
(time_data, freqs_data, masked_data) = load_hdf5_data(path_masked_data)

# sorted data to avoid issue later

sorted_time, sorted_data = zip(*sorted(zip(time_data, masked_data)))
del time_data, masked_data
time_data, masked_data = numpy.array(sorted_time), numpy.array(sorted_data)


for coordinate_system in coordinate_system_table:
    
    i_coordinate_system = i_coordinate_system+1

    (date_ephem_all_orbit,x_all_orbit,y_all_orbit,z_all_orbit) = polar_ephemeris_load_xyz(data_path_ephemeris, time_view_start_all_orbit, time_view_end_all_orbit,coordinate_system=coordinate_system)

#Interpolating the ephemeris data so it lines up with the flux data
    masked_data_all_orbit = masked_data[(time_data > date_ephem_all_orbit[0]) & (time_data < date_ephem_all_orbit[-1]),:]
    time_data_all_orbit = time_data[(time_data > date_ephem_all_orbit[0]) & (time_data < date_ephem_all_orbit[-1])]

    x_all_orbit = numpy.array(numpy.interp(time_data_all_orbit, date_ephem_all_orbit, x_all_orbit))
    y_all_orbit = numpy.array(numpy.interp(time_data_all_orbit, date_ephem_all_orbit, y_all_orbit))
    z_all_orbit = numpy.array(numpy.interp(time_data_all_orbit, date_ephem_all_orbit, z_all_orbit))
    date_ephem_all_orbit = numpy.array(numpy.interp(time_data_all_orbit, date_ephem_all_orbit, date_ephem_all_orbit))


# Normalizing @ 1 AU
    RE = 6371.0
    AU = 149597870.700 # 1 astronomical unit
    distance = numpy.sqrt(x_all_orbit**2+y_all_orbit**2+z_all_orbit**2)
    for i_freq in range(len(freqs_data)):
        masked_data_all_orbit[:, i_freq] = masked_data_all_orbit[:,i_freq]*(distance*RE/AU)**2


    integrated_flux = []
    #for t in tqdm(time_data_first_orbit):
    #    colours_first_orbit.append(masked_data_first_orbit[time_data_first_orbit == t].sum())
    #for t in tqdm(time_data_last_orbit):
    #    colours_last_orbit.append(masked_data_last_orbit[time_data_last_orbit == t].sum())
    for t in tqdm(time_data_all_orbit):
        integrated_flux.append(masked_data_all_orbit[time_data_all_orbit == t].sum())







100%|██████████| 224616/224616 [01:38<00:00, 2276.95it/s]


In [9]:
rho = np.sqrt(x_all_orbit**2+y_all_orbit**2)
for x_index in range(len(x_all_orbit)):
    if x_all_orbit[x_index] < 0:
        rho[x_index] = -1*rho[x_index]

In [23]:

integrated_flux_log = np.log10(integrated_flux)

plot_trajectory_magnetic_equatorial_plane(rho,z_all_orbit, integrated_flux_log,coordinate_system, save_html = True, save_png = True, show_plot = False)

<ipython-input-23-e3592faa57e1>:1: RuntimeWarning:

divide by zero encountered in log10



In [ ]:

#integrated_flux_log = np.log10(integrated_flux)
fig, ax = plt.subplots()
ax.set_xlim(-5,5)
ax.set_ylim(-2,+10)
ax.axhline(0., linestyle='--', alpha=0.5, color='grey', zorder=0)
ax.axvline(0., linestyle='--', alpha=0.5, color='grey', zorder=0)
    
vmin = np.quantile(integrated_flux,0.35)
vmax = np.quantile(integrated_flux,0.90)

plot_colourline(rho, z_all_orbit, numpy.array(integrated_flux),vmin=vmin,vmax=vmax)

angle_circle = np.linspace(0,2*np.pi,150)
radius_circle = 1
x_circle = radius_circle * np.cos(angle_circle)
y_circle = radius_circle * np.sin(angle_circle)
ax.plot(x_circle,y_circle,color='black')

cmap=plt.get_cmap('viridis')
scale_intensity = colors.LogNorm(vmin=vmin,vmax=vmax)
sm=cm.ScalarMappable(cmap=cmap,norm=scale_intensity)
sm.set_array([])
plt.colorbar(sm)

ax.set_title('Integrated intensity @ 1 AU')
ax.set_xlabel(r'$\rho = \sqrt{x^2+y^2}$ ($R_{E}$)')
ax.set_ylabel(r'$z$ ($R_{E}$)')
plt.tight_layout()

plt.savefig("magnetic_equatorial_plane_GSM.png")

plt.show()

    